# Qwen2-VL Model Preparation with OpenVINO

This notebook downloads Qwen2-VL-2B-Instruct, exports it to OpenVINO IR format,
and creates an INT4 quantized variant for CPU inference.

In [ ]:
from pathlib import Path

MODEL_ID = "Qwen/Qwen2-VL-2B-Instruct"

MODELS_DIR = Path("../models")

FP16_DIR = MODELS_DIR / "qwen2-vl-2b-instruct-fp16"
INT4_DIR = MODELS_DIR / "qwen2-vl-2b-instruct-int4"

# Ensure output directories exist
MODELS_DIR.mkdir(parents=True, exist_ok=True)

print(f"Model: {MODEL_ID}")
print(f"Output directories:")
print(f"  FP16: {FP16_DIR}")
print(f"  INT4: {INT4_DIR}")

## Export to OpenVINO IR (FP16)

Uses `optimum-cli` to download the model from HuggingFace and export to OpenVINO IR format.
The HuggingFace cache is shared across all exports, so the subsequent INT4 export won't re-download.

In [2]:
from pathlib import Path

FP16_DIR = Path("../models/qwen2-vl-2b-instruct-fp16")

if FP16_DIR.exists():
    print(f"FP16 model already exists at {FP16_DIR}, skipping export.")
else:
    print(f"Exporting {MODEL_ID} to OpenVINO IR (FP16)...")
    !optimum-cli export openvino \
        --model {MODEL_ID} \
        --task image-text-to-text \
        --weight-format fp16 \
        --trust-remote-code \
        {FP16_DIR}
    print(f"FP16 model saved to {FP16_DIR}")

FP16 model already exists at ../models/qwen2-vl-2b-instruct-fp16, skipping export.


## INT4 Quantization

In [4]:
if INT4_DIR.exists():
    print(f"INT4 model already exists at {INT4_DIR}, skipping.")
else:
    print(f"Exporting {MODEL_ID} to OpenVINO IR with INT4 weight compression...")
    !optimum-cli export openvino \
        --model {MODEL_ID} \
        --task image-text-to-text \
        --weight-format int4 \
        --group-size 64 \
        --sym \
        --trust-remote-code \
        {INT4_DIR}
    print(f"INT4 model saved to {INT4_DIR}")

Exporting Qwen/Qwen2-VL-2B-Instruct to OpenVINO IR with INT4 weight compression...
/home/shashank/projects/intel/llm-chat-app/.venv/lib/python3.12/site-packages/optimum/intel/openvino/modeling_base.py:590: SyntaxWarning: invalid escape sequence '\.'
  pattern=cls._search_pattern if not kwargs.get("from_onnx", False) else ".*\.onnx$",
`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:00<00:00, 20.85it/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.
/home/shashank/projects/intel/llm-chat-app/.venv/lib/

## Model Size Comparison

In [ ]:
def get_dir_size_mb(path: Path) -> float:
    """Calculate total size of all files in a directory in MB."""
    if not path.exists():
        return 0.0
    total = sum(f.stat().st_size for f in path.rglob("*") if f.is_file())
    return total / (1024 * 1024)

print("Model Size Comparison")
print("=" * 40)

for label, path in [("FP16", FP16_DIR), ("INT4", INT4_DIR)]:
    size = get_dir_size_mb(path)
    status = f"{size:.1f} MB" if size > 0 else "Not found"
    print(f"{label:>6}: {status}")

fp16_size = get_dir_size_mb(FP16_DIR)
int4_size = get_dir_size_mb(INT4_DIR)

if fp16_size > 0 and int4_size > 0:
    print(f"\nCompression ratio (vs FP16):")
    print(f"  INT4: {fp16_size / int4_size:.2f}x smaller")

## Run model inference

In [6]:
import openvino_genai as ov_genai

print(f"Loading INT4 model from {INT4_DIR}...")
pipe = ov_genai.VLMPipeline(str(INT4_DIR), "CPU")
print("Model loaded successfully!")

Loading INT4 model from ../models/qwen2-vl-2b-instruct-int4...
Model loaded successfully!


In [9]:
# Text-only test
print("Text-only test:")
print("-" * 40)

config = ov_genai.GenerationConfig()
config.max_new_tokens = 100

history = ov_genai.ChatHistory()
history.append({"role": "user", "content": "What is the capital of France?"})

result = pipe.generate(history, generation_config=config)
history.append({"role": "assistant", "content": result.texts[0]})

print(f"Q: What is the capital of France?")
print(f"A: {result.texts[0]}")

Text-only test:
----------------------------------------
Q: What is the capital of France?
A: The capital of France is Paris.


In [ ]:
# Image test
import numpy as np
import openvino as ov
from PIL import Image

def preprocess_image(image, max_dim=28*4):
    """Resize image so its longest side is at most *max_dim* pixels."""
    image = image.convert("RGB")
    w, h = image.size
    if max(w, h) <= max_dim:
        return image
    scale = max_dim / max(w, h)
    return image.resize((int(w * scale), int(h * scale)), Image.LANCZOS)

def pil_to_ov_tensor(image):
    """Convert a PIL Image to an OpenVINO Tensor (with preprocessing)."""
    return ov.Tensor(np.array(preprocess_image(image)))

def streamer(token: str) -> bool:
    print(token, end="", flush=True)
    return False

print("Image test:")
print("-" * 40)

test_image = Image.open("bunny.png")
ov_image = pil_to_ov_tensor(test_image)

history = ov_genai.ChatHistory()
history.append({"role": "user", "content": "Describe what you see in this image."})

print(f"Q: Describe what you see in this image.")
print(f"A: ", end="")
result = pipe.generate(history, images=[ov_image], generation_config=config, streamer=streamer)
history.append({"role": "assistant", "content": result.texts[0]})


Image test:
----------------------------------------
Q: Describe what you see in this image.
A: The image shows a brown rabbit sitting on a grassy surface. The rabbit appears to be in a natural setting, possibly a garden or a backyard. The background is blurred, which helps to focus attention on the rabbit.